In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader

In [3]:


# Define a simple CNN model for colorization
class ColorizationCNN(nn.Module):
    def __init__(self):
        super(ColorizationCNN, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 3, kernel_size=3, stride=1, padding=1),
            nn.Sigmoid()  # To normalize outputs between 0 and 1
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x



In [4]:
# Prepare dataset (CIFAR-10 as a proxy for colorization task)
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
    transforms.ToTensor()
])

color_transform = transforms.Compose([
    transforms.ToTensor()
])

gray_dataset = CIFAR10(root="./data", train=True, transform=transform, download=True)
color_dataset = CIFAR10(root="./data", train=True, transform=color_transform, download=True)

data_loader = DataLoader(list(zip(gray_dataset, color_dataset)), batch_size=32, shuffle=True)

# Initialize model, loss, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ColorizationCNN().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)



100%|██████████| 170498071/170498071 [00:06<00:00, 26445933.82it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [5]:
# Train the model
epochs = 30
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for (gray, color) in data_loader:
        gray_images = gray[0].unsqueeze(1).to(device)  # Grayscale input with channel dimension
        color_images = color[0].to(device)  # Target color output

        optimizer.zero_grad()
        outputs = model(gray_images)
        loss = criterion(outputs, color_images)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(data_loader):.4f}")

# Save the model
torch.save(model.state_dict(), "colorization_model.pth")
print("Model saved as colorization_model.pth")


RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [32, 1, 1, 32, 32]

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader

# Define a simple CNN model for colorization
class ColorizationCNN(nn.Module):
    def __init__(self):
        super(ColorizationCNN, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 3, kernel_size=3, stride=1, padding=1),
            nn.Sigmoid()  # To normalize outputs between 0 and 1
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Prepare dataset (CIFAR-10 as a proxy for colorization task)
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
    transforms.ToTensor()
])

color_transform = transforms.Compose([
    transforms.ToTensor()
])

gray_dataset = CIFAR10(root="./data", train=True, transform=transform, download=True)
color_dataset = CIFAR10(root="./data", train=True, transform=color_transform, download=True)

data_loader = DataLoader(list(zip(gray_dataset, color_dataset)), batch_size=32, shuffle=True)

# Initialize model, loss, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ColorizationCNN().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


# Save the model
torch.save(model.state_dict(), "colorization_model.pth")
print("Model saved as colorization_model.pth")


Files already downloaded and verified
Files already downloaded and verified


AttributeError: 'list' object has no attribute 'unsqueeze'

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
import torchvision
from torchvision import transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import Dataset
import cv2

# Define a simple colorization model (U-Net style)
class ColorizationModel(nn.Module):
    def __init__(self):
        super(ColorizationModel, self).__init__()
        # Encoder
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.fc = nn.Linear(256 * 32 * 32, 3 * 32 * 32)  # output 3 channels (RGB)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = torch.relu(self.conv3(x))
        x = x.view(x.size(0), -1)  # Flatten for fully connected layer
        x = self.fc(x)
        x = x.view(x.size(0), 3, 32, 32)  # Reshape to 3 channels
        return x

# CIFAR-10 dataset and dataloaders
transform = transforms.Compose([transforms.Grayscale(num_output_channels=1),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.5], std=[0.5])])

train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Initialize the model, loss, and optimizer
model = ColorizationModel().cuda() if torch.cuda.is_available() else ColorizationModel()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)




Files already downloaded and verified


In [12]:
# Train the model for 25 epochs
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, _ in train_loader:
        images = images.cuda() if torch.cuda.is_available() else images
        optimizer.zero_grad()

        # Forward pass
        output = model(images)
        loss = criterion(output, images)  # comparing colorized output with input

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

Epoch [1/10], Loss: 0.0143
Epoch [2/10], Loss: 0.0008
Epoch [3/10], Loss: 0.0005
Epoch [4/10], Loss: 0.0005
Epoch [5/10], Loss: 0.0004
Epoch [6/10], Loss: 0.0004
Epoch [7/10], Loss: 0.0003
Epoch [8/10], Loss: 0.0003
Epoch [9/10], Loss: 0.0003
Epoch [10/10], Loss: 0.0003


In [13]:
# Save the model's state_dict (weights and parameters)
torch.save(model.state_dict(), "colorization_model.pth")
